In [36]:
# imports
import os
import sys
import re
from math import pi
from datetime import datetime, timedelta
from dotmap import DotMap
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import panel as pn
from panel.template import DarkTheme
from bokeh.io import output_file, show
from bokeh.models import (ColumnDataSource, HoverTool, Label, Legend, 
                          Div, LabelSet, FuncTickFormatter, CustomJS)
from bokeh.models.widgets import Panel, Tabs
from bokeh.models.tickers import FixedTicker
from bokeh.plotting import figure
from bokeh.transform import dodge, cumsum
from bokeh.layouts import row

In [37]:
def get_config():
    '''
    Function that gets the configuration of personalized variable definitions
    '''
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

In [38]:
def get_styling():
    css = '''
    :root {
        --mdc-theme-primary: rgba(0, 168, 65, 0.82) !important;
    }
    .bk-root .bk, .bk-root .bk:before, .bk-root .bk:after {
        font-family: Avenir Next, Helvetica !important;
    }
    
    .selectDiv select {
      border-color: rgba(0, 168, 65, 0.82) !important;
      border-radius: 3px !important;
      background-image: url(data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pg0KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDE2LjAuMCwgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPg0KPCFET0NUWVBFIHN2ZyBQVUJMSUMgIi0vL1czQy8vRFREIFNWRyAxLjEvL0VOIiAiaHR0cDovL3d3dy53My5vcmcvR3JhcGhpY3MvU1ZHLzEuMS9EVEQvc3ZnMTEuZHRkIj4NCjxzdmcgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB4PSIwcHgiIHk9IjBweCINCgkgd2lkdGg9IjI4NC45MjlweCIgaGVpZ2h0PSIyODQuOTI5cHgiIHZpZXdCb3g9IjAgMCAyODQuOTI5IDI4NC45MjkiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDI4NC45MjkgMjg0LjkyOTsiDQoJIHhtbDpzcGFjZT0icHJlc2VydmUiPg0KPGc+DQoJPHBhdGggZD0iTTI4Mi4wODIsNzYuNTExbC0xNC4yNzQtMTQuMjczYy0xLjkwMi0xLjkwNi00LjA5My0yLjg1Ni02LjU3LTIuODU2Yy0yLjQ3MSwwLTQuNjYxLDAuOTUtNi41NjMsMi44NTZMMTQyLjQ2NiwxNzQuNDQxDQoJCUwzMC4yNjIsNjIuMjQxYy0xLjkwMy0xLjkwNi00LjA5My0yLjg1Ni02LjU2Ny0yLjg1NmMtMi40NzUsMC00LjY2NSwwLjk1LTYuNTY3LDIuODU2TDIuODU2LDc2LjUxNUMwLjk1LDc4LjQxNywwLDgwLjYwNywwLDgzLjA4Mg0KCQljMCwyLjQ3MywwLjk1Myw0LjY2MywyLjg1Niw2LjU2NWwxMzMuMDQzLDEzMy4wNDZjMS45MDIsMS45MDMsNC4wOTMsMi44NTQsNi41NjcsMi44NTRzNC42NjEtMC45NTEsNi41NjItMi44NTRMMjgyLjA4Miw4OS42NDcNCgkJYzEuOTAyLTEuOTAzLDIuODQ3LTQuMDkzLDIuODQ3LTYuNTY1QzI4NC45MjksODAuNjA3LDI4My45ODQsNzguNDE3LDI4Mi4wODIsNzYuNTExeiIvPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPC9zdmc+DQo=) !important;
      background-size: 10px 10px !important;
    }
    
    .selectDiv label {
        font-weight: bold;
    }
    
    .inputElement input[type="checkbox"] {
        width: 15px;
        height: 15px;
        border: 1px solid rgba(0, 168, 65, 0.82);
        border-radius: 2px;
        -webkit-appearance: none;
        -webkit-transition: box-shadow 200ms;
    }
    .inputElement input[type="checkbox"]:checked {
        background-color: rgba(0, 168, 65, 0.82);
    }
    '''
    return css

In [39]:
def preprocess_intake_files(self, subject):
    '''
    Function that preprocesses the intakes files from the given subject to the correct formattings
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    
    Returns
    --------------
    list
        baseline_filename    the filename of the created baseline intake file
        vegan_filename       the filename of the created vegan intake file
    '''
    
    days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    baseline = ["week1", "week2"]
    header = "Day,Date,Year,Calories,Fat,Saturated fat,Carbs,Fiber,Sugar,Protein,Sodium,Cholesterol,Potassium\n"
    
    food_intake_files_path = self.config['food_intake_files_path']
    processed_files_path = food_intake_files_path + '/processed'
    if not os.path.exists(processed_files_path):
        os.makedirs(processed_files_path)
    
    baseline_filename = processed_files_path + '/nutrition_baseline_subject_'+ subject +'.csv'
    nutrition_baseline = open(baseline_filename, 'w')
    nutrition_baseline.write(header)
    
    vegan_filename = processed_files_path + '/nutrition_vegan_subject_'+ subject +'.csv'
    nutrition_vegan = open(vegan_filename, 'w')
    nutrition_vegan.write(header)
    
    with nutrition_baseline as n_baseline, nutrition_vegan as n_vegan:
        for file in os.scandir(food_intake_files_path):
            filename = file.path
            # only use the file from the given subject
            if (('subject_' + subject in filename) and filename.lower().endswith('.csv')):
                file = open(file, 'r')
                with file as food_intake:
                    # start at line 10 with reading the food log
                    for i in range(9):
                        food_intake, next(food_intake)
                    for line in food_intake:
                        # replace double quotes with single quotes
                        index = 1 if '""' in line else 0
                        line = line.strip().replace('""', '"')[index:].rstrip('"')
                        # get the nutrition details per day
                        if line.split(',')[0].strip('"').lower() in days:
                            if any(week in filename.lower() for week in baseline):
                                n_baseline.write(line + '\n')
                            else:
                                n_vegan.write(line + '\n')
    
    return [baseline_filename, vegan_filename]

In [40]:
def create_nutrition_tables(self, subject):
    '''
    Function that creates the food intake data tables for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    
    Returns
    --------------
    list
        nutrition_data    food intake data table
    '''
    
    nutrition_figures = []
    
    subject_data = preprocess_intake_files(self, subject)
    base_data = pd.read_csv(subject_data[0], sep=',')
    vegan_data = pd.read_csv(subject_data[1], sep=',')

    convert_columns = ['Calories', 'Fat', 'Saturated fat', 'Carbs', 
                       'Fiber', 'Sugar', 'Protein', 
                       'Sodium', 'Cholesterol', 'Potassium']

    # convert commas to points as delimiter and convert to numeric values
    base_data[convert_columns] = base_data[convert_columns].replace(',', '.', regex=True)
    base_data[convert_columns] = base_data[convert_columns].apply(pd.to_numeric, errors='coerce')
    base_data['Calories'] = base_data['Calories'].astype(float)
    vegan_data[convert_columns] = vegan_data[convert_columns].replace(',', '.', regex=True)
    vegan_data[convert_columns] = vegan_data[convert_columns].apply(pd.to_numeric, errors='coerce')
    vegan_data['Calories'] = vegan_data['Calories'].astype(float)
    
    # convert sodium, cholesterol and potassium to grams (they are initially in mg)
    base_data[convert_columns[7:]] = base_data[convert_columns[7:]] / 100
    vegan_data[convert_columns[7:]] = vegan_data[convert_columns[7:]] / 100
    
    # set the average saturated fat of the week to '0' if it is higher than the fat value
    base_data.loc[(base_data['Saturated fat'] > base_data['Fat']), 'Saturated fat'] = 0
    vegan_data.loc[(vegan_data['Saturated fat'] > vegan_data['Fat']), 'Saturated fat'] = 0
    
    # add the diet definition column
    base_data['Diet'] = 'Normal'
    vegan_data['Diet'] = 'Vegan'
    
    # concat diets and set date as index column
    nutrition_data = pd.concat([base_data, vegan_data])
    nutrition_data['Date'] = nutrition_data['Date'].astype(str) + ' ' + nutrition_data['Year'].astype(str)
    nutrition_data['Date'] = pd.to_datetime(nutrition_data['Date'], format=' %B %d %Y')
    nutrition_data.drop(['Year'], inplace=True, axis=1)
    nutrition_data.set_index('Date', inplace=True)
    nutrition_data.sort_index(inplace=True)
    
    return nutrition_data

In [41]:
def create_nutrition_boxplot(subject, nutrition_data):
    '''
    Function that creates the nutrition boxplot for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : DataFrame
        The food intake data table of the given subject
    
    Returns
    --------------
    figure
        The created nutrition boxplot visualization
    '''
    tabs = []
    # graph labels and positions
    plot_definitions = ['Calories', 'Macronutrients', 'Carbohydrates', 'Minerals']
    boxplot_data = get_labels_and_positions(plot_definitions)
    
    for group, data in zip(plot_definitions, boxplot_data):
        bar_labels = data[0]; base_positions = data[1]; vegan_positions = data[2]; label_positions = data[3]
        graph_labels = data[4]; outlier_plot_positions = data[5]; outlier_plot_positions_v = data[6]
    
        vegan = nutrition_data.loc[nutrition_data['Diet'] == 'Vegan'][bar_labels]
        baseline = nutrition_data.loc[nutrition_data['Diet'] == 'Normal'][bar_labels]

        # transform both dataframes to 2 column dataframe
        baseline_df = pd.DataFrame(columns=['group', 'value'])
        vegan_df = pd.DataFrame(columns=['group', 'value'])
        for label in bar_labels:
            tmp = pd.DataFrame(columns=['group', 'value'])
            tmp_v = pd.DataFrame(columns=['group', 'value'])
            tmp['value'] = baseline[label]
            tmp_v['value'] = vegan[label]
            tmp['group'] = label
            tmp_v['group'] = label
            baseline_df = pd.concat([baseline_df, tmp])
            vegan_df = pd.concat([vegan_df, tmp_v])
        
        # find the quartiles and IQR for each category
        groups = baseline_df.groupby('group', sort=False)
        groups_v = vegan_df.groupby('group', sort=False)
        q1 = groups.quantile(q=0.25)
        q1_v = groups_v.quantile(q=0.25)
        q2 = groups.quantile(q=0.5)
        q2_v = groups_v.quantile(q=0.5)
        q3 = groups.quantile(q=0.75)
        q3_v = groups_v.quantile(q=0.75)
        iqr = q3 - q1
        iqr_v = q3_v - q1_v
        upper = q3 + 1.5*iqr
        upper_v = q3_v + 1.5*iqr_v
        lower = q1 - 1.5*iqr
        lower_v = q1_v - 1.5*iqr_v

        # get the outliers
        out = groups.apply(get_outliers, upper=upper, lower=lower).dropna()
        out_v = groups_v.apply(get_outliers, upper=upper_v, lower=lower_v).dropna()
        
        # prepare outlier data for plotting
        if not out.empty:
            outx = []
            outy = []
            for keys in out.index:
                if isinstance(out, pd.DataFrame):
                    outx.append(keys)
                    outy.append(out.loc[keys].iloc[0])
                else:
                    outx.append(keys[0])
                    outy.append(out.loc[keys[0]].loc[keys[1]])
        if not out_v.empty:
            outx_v = []
            outy_v = []
            for keys in out_v.index:
                if isinstance(out_v, pd.DataFrame):
                    outx_v.append(keys)
                    outy_v.append(out_v.loc[keys].iloc[0])
                else:
                    outx_v.append(keys[0])
                    outy_v.append(out_v.loc[keys[0]].loc[keys[1]])

        if group == 'Calories':
            y_range = (0, 6000)
            plot_width = 500
            title = "Boxplot representing caloric information of subject {}".format(subject)
            y_axis_label = "Calories (kcal)"
        else:
            y_range = (0, 500) if group == 'Macronutrients' else (0, 300)
            plot_width = 500
            title = "Boxplot representing {} of subject {} ".format(group.lower(), subject)
            y_axis_label = "Grams (g)"
        
        p = figure(tools="", toolbar_location=None, 
                   plot_height=500, plot_width=plot_width, y_range=y_range, title=title,
                   x_axis_label="Nutrient", y_axis_label=y_axis_label)

        # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
        qmin = groups.quantile(q=0.00)
        qmin_v = groups_v.quantile(q=0.00)
        qmax = groups.quantile(q=1.00)
        qmax_v = groups_v.quantile(q=1.00)
        upper.value = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'value']),upper.value)]    
        upper_v.value = [min([x,y]) for (x,y) in zip(list(qmax_v.loc[:,'value']),upper_v.value)]
        lower.value = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'value']),lower.value)]
        lower_v.value = [max([x,y]) for (x,y) in zip(list(qmin_v.loc[:,'value']),lower_v.value)]

        # stems
        p.segment(base_positions, upper.value, base_positions, q3.value, line_color="black")
        p.segment(base_positions, lower.value, base_positions, q1.value, line_color="black")
        p.segment(vegan_positions, upper_v.value, vegan_positions, q3_v.value, line_color="black")
        p.segment(vegan_positions, lower_v.value, vegan_positions, q1_v.value, line_color="black")

        # boxes
        p.vbar(base_positions, 0.4, q2.value, q3.value, fill_color="#abdfff", line_color="black", legend_label="Normal diet")
        p.vbar(base_positions, 0.4, q1.value, q2.value, fill_color="#abdfff", line_color="black")
        p.vbar(vegan_positions, 0.4, q2_v.value, q3_v.value, fill_color="#ceffc4", line_color="black", legend_label="Vegan diet")
        p.vbar(vegan_positions, 0.4, q1_v.value, q2_v.value, fill_color="#ceffc4", line_color="black")

        # whiskers
        p.rect(base_positions, lower.value, 0.2, 0.01, line_color="black")
        p.rect(base_positions, upper.value, 0.2, 0.01, line_color="black")
        p.rect(vegan_positions, lower_v.value, 0.2, 0.01, line_color="black")
        p.rect(vegan_positions, upper_v.value, 0.2, 0.01, line_color="black")

        # outliers
        if not out.empty:
            p.circle([outlier_plot_positions[x] for x in outx], outy, size=3, color="#F38630", fill_alpha=0.6)
        if not out_v.empty:
            p.circle([outlier_plot_positions_v[x] for x in outx_v], outy_v, size=3, color="#F38630", fill_alpha=0.6)

        p.xgrid.grid_line_color = None
        p.xaxis.major_label_text_font_size = "10px"
        p.xaxis.major_label_orientation = 0.5
        p.xaxis.ticker = label_positions

        # set labels at the tick positions
        p.xaxis.formatter = FuncTickFormatter(code="""
                var mapping = {};
                return mapping[tick];
            """.format(graph_labels))
        
        tab = Panel(child=p, title=group)
        tabs.append(tab)
    
    tabs = Tabs(tabs=tabs)
    return tabs

def get_labels_and_positions(columns):
    '''
    Function that creates the labels and position of the different nutrition boxplots
    '''
    ['Calories', 'Macronutrients', 'Carbohydrates', 'Minerals']
    boxplot_data = []
    for label in columns:
        if label == 'Calories':
            bar_labels = ['Calories']
            base_positions = [0.5]
        elif label == 'Macronutrients':
            bar_labels = ['Carbs', 'Protein', 'Fat', 'Saturated fat']
            base_positions = [0.5, 2.5, 4.5, 6.5]
        elif label == 'Carbohydrates':
            bar_labels = ['Fiber', 'Sugar']
            base_positions = [0.5, 2.5]
        elif label == 'Minerals':
            bar_labels = ['Sodium', 'Cholesterol', 'Potassium']
            base_positions = [0.5, 2.5, 4.5]
        
        vegan_positions = [pos+0.6 for pos in base_positions]
        label_positions = [pos+0.3 for pos in base_positions]
        graph_labels = str(dict(zip(label_positions, bar_labels)))
        outlier_plot_positions = dict(zip(bar_labels, base_positions))
        outlier_plot_positions_v = dict(zip(bar_labels, vegan_positions))
        boxplot_data.append([bar_labels, base_positions, vegan_positions, label_positions, 
                                    graph_labels, outlier_plot_positions, outlier_plot_positions_v])
    return boxplot_data

def get_outliers(group, upper, lower):
    '''
    Function that finds the outliers for each category
    '''
    
    cat = group.name
    return group[(group.value > upper.loc[cat]['value']) | (group.value < lower.loc[cat]['value'])]['value']

In [42]:
def create_average_nutrition_figure(subject, nutrition_data):
    '''
    Function that creates the nutrition graph for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : DataFrame
        The food intake data table of the given subject
    
    Returns
    --------------
    figure
        The created nutrition graph visualization
    '''
    
    # calcuate the average nutritional values
    averages = nutrition_data.groupby(['Diet']).mean().round(2)
    
    bar_labels = ['Carbs', 'Protein', 'Fat', 
                 'Saturated fat', 'Fiber', 'Sugar', 
                 'Sodium', 'Cholesterol', 'Potassium']
    data_obj = {
            'labels': bar_labels,
            'Normal': averages[bar_labels].loc['Normal'],
            'Vegan': averages[bar_labels].loc['Vegan'],
            'None' : [0 for i in bar_labels]
        }
    source = ColumnDataSource(data=data_obj)
    
    p = figure(x_range=bar_labels, y_range=(0, 400), plot_height=500, plot_width=600,
               title="Average nutritional information subject " + subject + " in grams",
               x_axis_label="Nutrient", y_axis_label="Grams (g)", 
               toolbar_location=None, tools="")
    
    p.vbar(x=dodge('labels', -0.12, range=p.x_range), 
           top='Normal', name='Normal', width=0.2, 
           source=source, color="#abdfff", legend_label="Normal diet", line_color="#75cbff")

    # middle label
    p.vbar(x=dodge('labels',  0,  range=p.x_range), top='None', width=0.1, source=source)
    
    p.vbar(x=dodge('labels',  0.12,  range=p.x_range), 
           top='Vegan', name='Vegan', width=0.2, 
           source=source, color="#ceffc4", legend_label="Vegan diet", line_color="#8dff75")
    
    p.x_range.range_padding = 0.05
    p.xgrid.grid_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "vertical"
    p.xaxis.major_label_orientation = 0.5
    p.xaxis.major_label_text_font_size = "10px"
    
    p.add_tools(HoverTool(
        names = ['Normal', 'Vegan'],
        tooltips = [
            ('', '$name diet'),
            ('', '@$name{1.11} grams')
        ],
        mode = 'mouse',
        show_arrow = False,
        point_policy = 'follow_mouse'
    ))
    
    style = {
        'font-size': 'smaller',
        'margin-top': '20px'
    }
    div = Div(text="""
        <p><b>Average calories:</b><br>
        Normal diet: {} kcal <br>
        Vegan diet:  {} kcal</p>
        """.format(averages['Calories'].loc['Normal'], averages['Calories'].loc['Vegan']),
              width=200, height=100, style=style)
    div.css_classes = ["custom"]

    return row(p, div)

In [43]:
def create_nutrition_piecharts(subject, nutrition_data):
    '''
    Function that creates the food intake pie charts per week for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : list
        The food intake data table
    
    Returns
    --------------
    figure
        The tab-seperated intake pie charts
    '''
    
    tabs = []
    weekly_averages = get_weekly_averages(nutrition_data)
    week_starts = {0: '2020-10-12', 1: '2020-10-19', 2: '2020-10-26', 
                   3: '2020-11-02', 4: '2020-11-09', 5: '2020-11-16'}
    
    macro_colors = ['#266298', '#89c609', '#fc2a35']
    sug_fib_colors = ['#f9dda9', '#ac545c']
    micro_colors = ['#a21d22', '#f6871e']
    
    for i in range(0, len(weekly_averages.index)):
        macros = weekly_averages[['Carbs', 'Protein', 'Fat']].iloc[i]
        sug_fib = weekly_averages[['Sugar', 'Fiber']].iloc[i]
        micros = weekly_averages[['Sodium', 'Potassium']].iloc[i]

        p1 = get_piechart_data(macros, macro_colors, 'Average calorie composition')
        p2 = get_piechart_data(sug_fib, sug_fib_colors, 'Average sugar and fiber devision')
        p3 = get_piechart_data(micros, micro_colors, 'Average sodium and potassium devision')

        pie_charts = row(p1, p2, p3)
        tab = Panel(child=pie_charts, title="Week " + str(i+1))
        tabs.append(tab)
        
    tabs = Tabs(tabs=tabs)
    return tabs

In [44]:
def get_piechart_data(nutrients, nutrient_colors, chart_description):
    '''
    Function that creates a food intake pie chart for the given nutrients DataFrame
    
    Parameters
    --------------
    nutrients : DataFrame
        The DataFrame with the nutrient values
    nutrition_colors : list
        The nutrient colors list
    chart_description : string
        The title of the pie chart graph
    
    Returns
    --------------
    p
        The generated pie chart
    '''
    
    R = 0.35
    p_range = (-R * 1.1, R * 1.5)
    p_size = 350
    
    data = (pd.Series(nutrients.to_dict())
            .reset_index(name='value')
            .rename(columns={'index':'nutrient'})
            .assign(end_angle=lambda d: np.cumsum(d['value'] / d['value'].sum() * 2 * pi),
                    start_angle=lambda d: np.pad(d['end_angle'], (1, 0), mode='constant')[:-1],
                    label_x=lambda d: R * 0.9 * np.cos(d['start_angle']),
                    label_y=lambda d: R * 0.95 * np.sin(d['start_angle'])))
    data['percentage'] = (data['value']/data['value'].sum() * 100).round(2).astype(str) + '%'
    data['color'] = nutrient_colors[:len(nutrients)]

    p = figure(title=chart_description, toolbar_location=None,
               plot_height=p_size, plot_width=p_size,
               x_range=p_range, y_range=p_range,
               tools="")
    p.wedge(x=0, y=0, radius=R,
            start_angle='start_angle', end_angle='end_angle',
            line_color="white", fill_color='color', legend_field='nutrient', source=data)

    source = ColumnDataSource(data)
    labels = LabelSet(x='label_x', y='label_y', text='percentage',
                      angle='start_angle', source=source, render_mode='canvas', 
                      text_align='right', text_font_size="9pt", text_color='white')

    p.add_layout(labels)
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    p.legend.label_text_font_size = '9pt'
    
    return p

In [45]:
def get_weekly_averages(nutrition_data):
    '''
    Function that calculates the weekly average nutrient values
    '''
    
    nutrition_data['week_number'] = ((nutrition_data.Day.str.lower() == 'monday').cumsum())
    return nutrition_data.groupby('week_number').mean().round(2)

In [46]:
def create_nutrition_dataframe(nutrition_data):
    column_order = ['Carbs', 'Protein', 'Fat', 
                 'Saturated fat', 'Fiber', 'Sugar', 
                 'Sodium', 'Cholesterol', 'Potassium']
    
    weekly_df = nutrition_data.drop(columns=['Diet', 'week_number'])
    # add the mean and the median
    weekly_df.index = weekly_df.index.astype(str)
    weekly_df = pd.concat([weekly_df.round(2), weekly_df.mean().round(2).to_frame('Mean').T,
                        weekly_df.median().round(2).to_frame('Median').T])
    weekly_df.fillna('', inplace=True)
    weekly_df = pd.concat([weekly_df.iloc[:, 0:2], weekly_df[column_order].add_suffix(' (g)')], axis=1)
    
    return pn.widgets.DataFrame(weekly_df, autosize_mode='none', frozen_rows=-2,
                                width=1050, widths=86, height=350)

In [47]:
def create_nutrition_graphs(self):
    '''
    Function that creates and returns the nutrition data graphs
    '''
    subjects = self.subjects
    figures = {}
    intake_dfs = {}
    for subject in subjects:
        nutrition_data = create_nutrition_tables(self, subject)
        averages_figure = create_average_nutrition_figure(subject, nutrition_data)
        boxplots = create_nutrition_boxplot(subject, nutrition_data)
        pie_charts = create_nutrition_piecharts(subject, nutrition_data)
        sub_figures = [boxplots, averages_figure, pie_charts]
        figures[subject] = sub_figures
        intake_dfs[subject] = create_nutrition_dataframe(nutrition_data)
    return figures, intake_dfs

In [48]:
'''
NOTE
Please add mental state functionality in this block!
'''
def create_mental_state_graphs(self):
    filepath = self.config['mental_state_files_path']
    pass

In [87]:
'''
NOTE
Please add body composition functionality in this block!
'''

import panel as pn
import hvplot.pandas
from panel.interact import interact

def tabs(self, sub):
    filepath = self.config['body_comp_files_path']
    df = pd.read_excel( filepath+'/'+'df_BC.xlsx')
    #print(sub)
    #print(df)
    dft = df[df['Subject'] == sub]
    return pn.Tabs(
        ('Weight' , dft['Bodyweight (kg)'].hvplot(x= 'Days')),
        ('BMI', dft['BMI (score)'].hvplot(x= 'Days')),
        ('Bodyfat', dft['Bodyfat (%)'].hvplot(x= 'Days')),
        ('Muscle', dft['Skeletal muscle (%)'].hvplot(x= 'Days')))

def create_body_comp_graphs(self, subjects):
    tabss = {}
    for sub in subjects:
        tabss[sub] = tabs(self, sub)
    return tabss

In [50]:
'''
NOTE
Please add microbiota functionality in this block!
'''
def create_microbiota_graphs(self):
    filepath = self.config['microbiota_files_path']
    pass

In [91]:
class Dashboard:
    def __init__(self):
        self.config = get_config()
        self.subjects = ['A', 'B', 'C', 'D', 'E']
        self.measurements = ['Food intake', 'Body composition', 'Mental state', 'Microbiota']
        self.panel_options = {}
        self.create_panel()
    
    def create_panel(self):
        # retrieve the plots
        
        '''
        NOTE
        Plots are retrieved here. Interactivity is also controlled here. Please keep the specific plot 
        functionolity as much in your own functions as possible. 
        '''
        nutrition_data = create_nutrition_graphs(self)
        self.nutrition_figures = nutrition_data[0]
        self.nutrition_tables = nutrition_data[1]
        self.mental_state_data = create_mental_state_graphs(self)
        self.body_comp_data = create_body_comp_graphs(self, self.subjects)
        self.microbiota_data = create_microbiota_graphs(self)

        self.subjects_menu = pn.widgets.Select(name='Subject:', options=self.subjects, value='A',
                                              css_classes=['selectDiv'])
        self.subjects_menu.param.watch(self.select_subject, 'value')
        
        self.checkbox_group = pn.widgets.CheckBoxGroup(name='Measurement', options=self.measurements,
                                                      value=['Food intake'], css_classes=['inputElement'])
        self.checkbox_group.param.watch(self.toggle_plots, 'value')
        
        food_intake_options = ['Boxplots', 'Average', 'Weekly piecharts']
        self.intake_plot_options = pn.widgets.Select(name='Food intake data:', options=food_intake_options, 
                                                     value='Boxplots', css_classes=['selectDiv'])
        self.intake_plot_options.param.watch(self.select_plot, 'value')
        self.table_checkbox = pn.widgets.Checkbox(name='View nutrition details', css_classes=['inputElement'])
        self.table_checkbox.param.watch(self.handle_intake_tables, 'value')

        # create the options menu
        self.options_column = pn.Column(self.subjects_menu, self.checkbox_group, self.intake_plot_options, 
                                        self.table_checkbox, 
                                        margin=0, sizing_mode='stretch_both')

        # display subject A and food intake boxplot as default
        self.panel_options['subject'] = 'A'
        self.panel_options['intake_plot_index'] = 0
        self.panel_options['food_intake_displayed'] = True
        
        '''
        NOTE
        Please add your plots to this column and see how it looks
        '''
        self.figure_column = pn.Column(self.nutrition_figures['A'][0], self.body_comp_data['A'])
        
        self.template = pn.template.MaterialTemplate(title='Project Ve-gang')
        self.template.sidebar.append(self.options_column)
        self.template.main.append(self.figure_column)
        pn.serve(self.template)
    
    def select_subject(self, event):
        '''
        Function that handles the selection of a different subject
        '''
        if event.new:
            subject = event.new
            prev_subject = event.old
            self.panel_options['subject'] = subject
            self.panel_options['intake_plot_index'] = 0
            self.panel_options['food_intake_displayed'] = True
            self.subjects_menu.value = subject
            self.checkbox_group.value = ['Food intake']
            self.intake_plot_options.value = 'Boxplots'
            
            if self.nutrition_tables[prev_subject] in self.figure_column: 
                self.figure_column.remove(self.nutrition_tables[prev_subject])
                self.table_checkbox.value = False
            self.figure_column[0] = self.nutrition_figures[subject][0]
        
    def select_plot(self, event):
        '''
        Function that handles the selection of a different plot type
        '''
        if event.new:
            subject = self.panel_options['subject']
            
            if self.panel_options['food_intake_displayed']:
                self.panel_options['event'] = 'select_plot'
                if event.new == 'Boxplots':
                    index = 0
                elif event.new == 'Average':
                    index = 1
                elif event.new == 'Weekly piecharts':
                    index = 2
                
                self.panel_options['intake_plot_index'] = index
                # add the nutrition figure to the panel
                if len(self.figure_column) > 0:
                    self.figure_column.pop(0)
                self.figure_column.insert(0, self.nutrition_figures[subject][index])

    def handle_intake_tables(self, event):
        '''
        Toggle show the intake datatables 
        '''
        subject = self.panel_options['subject']
        # checked
        if event.new:
            self.figure_column.insert(1, self.nutrition_tables[subject])
        # unchecked
        else:
            self.figure_column.remove(self.nutrition_tables[subject])
    
    def toggle_plots(self, event):
        '''
        '''
        subject = self.panel_options['subject']
        if not 'Food intake' in event.new:
            self.panel_options['food_intake_displayed'] = False
            intake_plot_index = self.panel_options['intake_plot_index']
            self.figure_column.pop(0)
            self.options_column.remove(self.intake_plot_options)
            self.options_column.remove(self.table_checkbox)
        elif self.intake_plot_options not in self.options_column:
            self.panel_options['food_intake_displayed'] = True
            self.options_column.append(self.intake_plot_options)
            self.options_column.append(self.table_checkbox)
            self.select_plot(DotMap({'new': 'Boxplots'}))
        
        if not 'Body composition' in event.new:
            # todo - remove body compostion options
            pass
        if not 'Mental state' in event.new:
            # todo - remove mental state options
            pass
        if not 'Microbiota' in event.new:
            # todo - remove microbiota options
            pass

In [92]:
def main():
    pn.extension(raw_css=[get_styling()])
    Dashboard()

if __name__ == '__main__':
    main()

C:\Users\Behzad\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Launching server at http://localhost:58553


In [93]:
pwd

'C:\\Users\\Behzad'